In [1]:
# DS LIBS
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# TODO: we're def gonna need some geospatial libs

# GEOSPATIAL LIBS
from osgeo import gdal


# ML LIBS



In [6]:
# import data
foodlink_data = pd.read_csv(r'data/foodlink_data.csv')
mealmap_data = pd.read_excel(r'data/MMG2022_2020-2019Data_ToShare.xlsx')
gun_data = pd.read_csv(r'data/Rochester_NY_Shooting_Victims.csv')

### STEP 1) DATA CLEANING

#### 1.1: GUN VIOLENCE DATA

In [17]:
print("GUN VIOLENCE DATASET INFO")
print(gun_data.info())


GUN VIOLENCE DATASET INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4679 entries, 0 to 4678
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  4679 non-null   float64
 1   Y                  4679 non-null   float64
 2   ID                 4679 non-null   object 
 3   Case_Number        4679 non-null   object 
 4   Address            4679 non-null   object 
 5   Occurred_Date      4679 non-null   object 
 6   Occurred_Month     4679 non-null   int64  
 7   Occurred_Year      4679 non-null   int64  
 8   Crime_Type         4679 non-null   object 
 9   Multiple_Shooting  4679 non-null   object 
 10  Gender             4679 non-null   object 
 11  Race               4679 non-null   object 
 12  Ethnicity          4679 non-null   object 
 13  Victim_Age         4679 non-null   int64  
 14  Victim_Age_Band    4679 non-null   object 
 15  Latitude           4679 non-null   float64
 16

In [16]:
print('SOME QUICK SUMMARY STATS:')
print(gun_data.describe())

SOME QUICK SUMMARY STATS:
                 X            Y  Occurred_Month  Occurred_Year   Victim_Age  \
count  4679.000000  4679.000000     4679.000000    4679.000000  4679.000000   
mean    -77.615629    43.166353        6.754007    2011.602907    27.200898   
std       0.023427     0.016363        3.142948       6.711393     9.868082   
min     -77.672811    43.115253        1.000000    2000.000000     0.000000   
25%     -77.635262    43.154277        4.000000    2006.000000    20.000000   
50%     -77.619800    43.169335        7.000000    2012.000000    25.000000   
75%     -77.597176    43.178504        9.000000    2018.000000    32.000000   
max     -77.538345    43.259854       12.000000    2022.000000    89.000000   

          Latitude    Longitude     ObjectId  
count  4679.000000  4679.000000  4679.000000  
mean     43.166353   -77.615629  2340.000000  
std       0.016363     0.023427  1350.855285  
min      43.115253   -77.672811     1.000000  
25%      43.154277   -77.63

In [23]:
# PRELIM QUESTIONS
print("MODAL DATA:")
print(gun_data['Occurred_Date'].mode(), '\n')
print(gun_data['Occurred_Month'].mode(),'\n')
print(gun_data['Occurred_Year'].mode(),'\n')
print(gun_data['Victim_Age_Band'].mode(),'\n')
print(gun_data['Gender'].mode(),'\n')
print(gun_data['Race'].mode(),'\n')
print(gun_data['Address'].mode(),'\n')

MODAL DATA:
0    2020/09/19 04:00:00+00
Name: Occurred_Date, dtype: object 

0    7
Name: Occurred_Month, dtype: int64 

0    2021
Name: Occurred_Year, dtype: int64 

0    15-24
Name: Victim_Age_Band, dtype: object 

0    M
Name: Gender, dtype: object 

0    BLACK
Name: Race, dtype: object 

0    278 Pennsylvania Ave
Name: Address, dtype: object 

